# Example: Recursive Implementation of Fibonacci Sequence Calculation
In this example, we illustrate recursion concepts by benchmarking different implementations of the Fibonacci sequence computation using the [BenchmarkTools.jl package](https://github.com/JuliaCI/BenchmarkTools.jl).

* __Case 1: Vanilla loop-based implementation:__ First, we benchmark the time required to calculate the sequence $F_{0},\dots,F_{n}$ using the vanilla for-loop implementation of the `fibonacci(...)` function.
* __Case 2: Standard recursive implementation:__ Next, we'll benchmark a recursive implementation. The `fibonacci!(n::Int64, series::Dict{Int64, Int64})::Nothing` function is a mutating recursive function that computes the sequence $F_{0},\dots, F_{n}$ for a given $n$. The recursive sequence is stored in the `series::Dict{Int64, Int64}` argument, which is updated in place.
* __Case 3: Memoized recursive implementation:__ Lastly, we'll benchmark a recursive implementation that uses memoization. The `memoization_fibonacci!(n::Int64, series::Dict{Int64, Int64})::Nothing` function is a mutating recursive function that uses memoization to speed up the computation of the sequence $F_{0},\dots, F_{n}$ for a given $n$. The recursive sequence is stored in the `series::Dict{Int64, Int64}` argument.

We expect that the memoized recursive implementation will be significantly faster than the standard recursive implementation, especially for larger values of $n$. This is because memoization avoids redundant calculations by storing previously computed results. However, let's see!

Let's get started!
___

## Setup, Data, and Prerequisites
First, we set up the computational environment by including the `Include.jl` file and loading any needed resources.

> The [include command](https://docs.julialang.org/en/v1/base/base/#include) evaluates the contents of the input source file, `Include.jl`, in the notebook's global scope. The `Include.jl` file sets paths, loads required external packages, etc. For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/). 


Let's set up the computational environment.

In [1]:
include(joinpath(@__DIR__, "Include.jl")); # what is this doing?

In addition to standard Julia libraries, we'll also use [the `VLDataScienceMachineLearningPackage.jl` package](https://github.com/varnerlab/VLDataScienceMachineLearningPackage.jl). Check out [the documentation](https://varnerlab.github.io/VLDataScienceMachineLearningPackage.jl/dev/) for more information on the functions, types, and data used in this material.

### Implementations
We've implemented three versions of the Fibonacci sequence calculation that we will benchmark. These are contained in [the `Compute.jl` file](src/Compute.jl), which is included in [the `Include.jl` file](Include.jl).

* __Vanilla loop-based implementation:__ The `fibonacci(n::Int64)::Vector{Int64}` function computes the Fibonacci sequence using a simple for-loop approach.
* __Standard recursive implementation:__ The `fibonacci!(n::Int64, series::Dict{Int64, Int64})::Nothing` function computes the Fibonacci sequence using a standard recursive approach. This implementation uses a mutating recursive function that computes the sequence $F_{0},\dots, F_{n}$ for a given $n$. The recursive sequence is stored in the `series::Dict{Int64, Int64}` argument, which is updated in place.
* __Memoized recursive implementation:__ The `memoization_fibonacci!(n::Int64, series::Dict{Int64, Int64})::Nothing` function computes the Fibonacci sequence using a memoized recursive approach to avoid redundant calculations. This implementation uses a mutating recursive function that computes the sequence $F_{0},\dots, F_{n}$ for a given $n$. The recursive sequence is stored in the `series::Dict{Int64, Int64}` argument, which is updated in place.



### Constants
Let's set some constants that we will use in the notebook. See the comment next to the constant value for what the constant is, permissible values, units, etc.

In [2]:
n = 25; # compute the Fibonacci sequence from F0 to F25
correct_fibonacci_sequence = Dict(0 => 0, 1 => 1, 2 => 1, 3 => 2, 4 => 3, 5 => 5, 6 => 8, 7 => 13, 8 => 21, 9 => 34, 10 => 55, 11 => 89, 12 => 144, 13 => 233, 14 => 377, 15 => 610);

___

## Case 1: Test the for loop implementation of Fibonacci computation
Let's use the [BenchmarkTools.jl package](https://github.com/JuliaCI/BenchmarkTools.jl) to compute the average time required to calculate the sequence $F_{0},\dots,F_{n}$ using the vanilla implementation of the `fibonacci` function (for-loop-based implementation). However, before we benchmark the for loop implementation, let's check that it is correct by [using the `@test` macro exported by the `Test.jl` package](https://docs.julialang.org/en/v1/stdlib/Test/).

In [3]:
let

    # initialize -
    number_of_test_terms = 15; # we have n = 15 terms in the correct_fibonacci_sequence dictionary
    my_computed_sequence = fibonacci(number_of_test_terms);
    
    for i ∈ 0:number_of_test_terms
        @test my_computed_sequence[i] == correct_fibonacci_sequence[i];
    end
end

Now that we have verified correctness, let's benchmark the for-loop implementation of the Fibonacci sequence calculation.

> __Benchmarking:__ The [BenchmarkTools.jl package](https://github.com/JuliaCI/BenchmarkTools.jl) exports the [@benchmarkable macro](https://juliaci.github.io/BenchmarkTools.jl/stable/reference/#BenchmarkTools.@benchmarkable-Tuple), which computes a function's runtime and memory profile. It runs the function many times and returns statistical information about its performance. Check out [the BenchmarkTools.jl documentation](https://github.com/JuliaCI/BenchmarkTools.jl) to see what some of these features, e.g., `samples` versus `evaluations` or the `tune!` method, do.

How does the vanilla implementation perform? Let's find out!

In [4]:
result_basal = let
    n = 50; # local variable to compute the Fibonacci sequence from F0 to F50
    test_run_basal = @benchmarkable fibonacci($(n));
    tune!(test_run_basal)
    result_basal = run(test_run_basal)
end

BenchmarkTools.Trial: 10000 samples with 10 evaluations per sample.
 Range (min … max):  1.042 μs … 637.013 μs  ┊ GC (min … max):  0.00% … 99.26%
 Time  (median):     1.363 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   1.722 μs ±  12.535 μs  ┊ GC (mean ± σ):  17.56% ±  2.43%

         ▁▄█▆▄▂                                                
  ▂▂▂▃▄▄▇███████▇▅▄▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▂
  1.04 μs         Histogram: frequency by time        2.81 μs <

 Memory estimate: 5.94 KiB, allocs estimate: 12.

## Case 2: Test the recursive implementation of the Fibonacci computation
Next, let's benchmark a recursive implementation. The `fibonacci!(n::Int64, series::Dict{Int64, Int64})::Nothing` function is a mutating recursive function that computes the sequence $F_{0},\dots, F_{n}$ for a given $n$. The recursive sequence is stored in the `series::Dict{Int64, Int64}` argument. This takes advantage of [the mutating function behavior](https://docs.julialang.org/en/v1/manual/functions/#man-argument-passing) in Julia, which allows us to update the dictionary in place without returning a new dictionary.

Let's verify that the recursive implementation is correct by checking that it computes the Fibonacci sequence correctly for $F_{0},\dots,F_{n}$, where $n$ is the number of terms in the `correct_fibonacci_sequence` dictionary.

In [5]:
let

    # initialize -
    number_of_test_terms = 15; # we have n = 15 terms in the correct_fibonacci_sequence dictionary
    my_computed_sequence = Dict{Int64, Int64}(); # initialize an empty dictionary
    fibonacci!(number_of_test_terms, my_computed_sequence); # notice that we are not returning anything, we are mutating the dictionary in place
    
    # verify correctness - for terms 0 ... number_of_test_terms
    for i ∈ 0:number_of_test_terms
        @test my_computed_sequence[i] == correct_fibonacci_sequence[i];
    end
end

No testing explosions? Great! Then how does the recursive implementation perform relative to the baseline implementation of the Fibonacci computation?

In [6]:
result_recursive = let
    n = 25; # local variable to compute the Fibonacci sequence from F0 to F25
    result_dictionary = Dict{Int,Int}() # empty dictionary to store the Fibonacci sequence
    test_run_recursive = @benchmarkable fibonacci!($(n), $(result_dictionary))
    tune!(test_run_recursive)
    result_recursive = run(test_run_recursive)
end

BenchmarkTools.Trial: 4060 samples with 1 evaluation per sample.
 Range (min … max):  1.208 ms …  1.465 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     1.222 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.231 ms ± 23.552 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

  ▂ █▇▄▅▄▃▃▅▄▄▄▃▂▂▃▂▃▂▂▂▂▂▂▁▂▂▁▁▁▁▁  ▁▁                      ▂
  ███████████████████████████████████████▇▇▇▇▆▇▆▅▃▅▄▄▅▄▁▄▁▁▅ █
  1.21 ms      Histogram: log(frequency) by time     1.32 ms <

 Memory estimate: 0 bytes, allocs estimate: 0.

__Hmmmm.__ Wow! The recursive implementation is __significantly slower__ than the baseline implementation. This is likely due to the overhead of recursive function calls and the way the algorithm processes the computation, even though it stores intermediate results. Lesson learned: recursion is not always faster than iteration!

<div>
    <center>
      <img
        src="figs/Fig-Fibonacci-Recursive.svg"
        alt="memoized recursive implementation of Fibonacci sequence computation"
        height="200"
        width="400"
      />
    </center>
  </div>

## Case 3: Test the recursive implementation of the Fibonacci computation with memoization
Finally, let's benchmark a recursive Fibonacci function that uses memoization. The `memoization_fibonacci!(n::Int64, series::Dict{Int64, Int64})::Nothing` implementation is a mutating recursive function that uses memoization to speed up the computation of the sequence $F_{0},\dots, F_{n}$ for a given $n$. The recursive sequence is stored in the `series::Dict{Int64, Int64}` argument.

First, does this implementation do what we expect? Let's verify that it computes the Fibonacci sequence correctly for $F_{0},\dots,F_{n}$, where $n$ is the number of terms in the `correct_fibonacci_sequence` dictionary.

In [7]:
let

    # initialize -
    number_of_test_terms = 15; # we have n = 15 terms in the correct_fibonacci_sequence dictionary
    my_computed_sequence = Dict{Int64, Int64}(); # initialize an empty dictionary
    memoization_fibonacci!(number_of_test_terms, my_computed_sequence); # notice that we are not returning anything, we are mutating the dictionary in place
    
    # verify correctness - for terms 0 ... number_of_test_terms
    for i ∈ 0:number_of_test_terms
        @test my_computed_sequence[i] == correct_fibonacci_sequence[i];
    end
end

Does the inclusion of the memoization change the runtime (or memory allocation) profile of the recursive Fibonacci implementation?

In [8]:
result_recursive_memo = let
    n = 50; # local variable to compute the Fibonacci sequence from F0 to F50
    result_dictionary_memo = Dict{Int,Int}()
    test_run_recursive_memo = @benchmarkable memoization_fibonacci!($(n), $(result_dictionary_memo))
    tune!(test_run_recursive_memo)
    result_recursive_memo = run(test_run_recursive_memo)
end

BenchmarkTools.Trial: 10000 samples with 1000 evaluations per sample.
 Range (min … max):  7.375 ns … 26.958 ns  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     7.458 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.557 ns ±  0.906 ns  ┊ GC (mean ± σ):  0.00% ± 0.00%

    ▄  █ ▅  ▂  ▃ ▂  ▁  ▁            ▁ ▁                      ▁
  ▄▁█▁▁█▁█▁▁█▁▁█▁█▁▁█▁▁█▁▆▁▁▆▁▅▄▁▇▁▁█▁█▁▁▆▁▁▅▁▇▁▁▆▁▁▆▁▆▁▁▇▁▆ █
  7.38 ns      Histogram: log(frequency) by time     8.29 ns <

 Memory estimate: 0 bytes, allocs estimate: 0.

## Summary
So what did we learn from benchmarking these three different approaches to computing the Fibonacci sequence? First, recursion isn't always faster than iteration - our vanilla recursive implementation was significantly slower than the simple for-loop approach. 

However, memoization can dramatically improve recursive performance by storing previously computed results and avoiding redundant calculations. The memoized recursive implementation performed much better than the standard recursive version (and even the for-loop implementation!)

The key takeaway? Choose your algorithm wisely based on the problem structure, and remember that elegant recursive solutions sometimes need optimization techniques like memoization to be practically useful!